In [62]:
!pip install -U bert-extractive-summarizer
!pip install -U python-docx
!pip install -U transformers
!pip install -U textract
!apt-get install antiword

Requirement already up-to-date: bert-extractive-summarizer in /usr/local/lib/python3.6/dist-packages (0.6.1)
Requirement already up-to-date: python-docx in /usr/local/lib/python3.6/dist-packages (0.8.10)
Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.2.2)
     |████████████████████████████████| 5.6MB 6.1MB/s 
     |████████████████████████████████| 8.9MB 30.4MB/s 
     |████████████████████████████████| 32.8MB 101kB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 112kB 35.8MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 112kB 38.1MB/s 
     |████████████████████████████████| 13.7MB 264kB/s 
     |████████████████████████████████| 153kB 47.6MB/s 
     |████████████████████████████████| 112kB 59.0MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.18-cp36-none-any.whl size=275707 

In [2]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-02-07 16:43:50--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.242, 5.45.205.243, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.242|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-02-07 16:43:51--  http://cache-mskm902.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)... 5.45.220.12, 2a02:6b8:0:2002::13
Connecting to cache-mskm902.cdn.yandex.net (cache-mskm902.cdn.yandex.net)|5.45.220.12|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [3]:
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE
!wget https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkrText.csv?token=AITWEHZGZ5SB7PHMJA3YNTDAFFJOM

--2021-02-07 16:43:54--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30447 (30K) [text/plain]
Saving to: ‘people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA’

people.csv?token=AI 100%[===================>]  29.73K  --.-KB/s    in 0.003s  

2021-02-07 16:43:54 (10.1 MB/s) - ‘people.csv?token=AITWEHYTYLFFCBVPMPPERTTAETPYA’ saved [30447/30447]

--2021-02-07 16:43:54--  https://raw.githubusercontent.com/Samoed/CourseWork2/main/data/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

In [4]:
import requests
import pandas as pd
from docx import Document
from summarizer import Summarizer
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
import string
from pymystem3 import Mystem
import re
import math

import textract

nltk.download('stopwords')
russian_stopwords = stopwords.words("russian")
russian_stopwords.append('рис')
russian_stopwords.append('т.к.')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
fullText = ""
for para in document.paragraphs:
    fullText+=(para.text)+" "
print(fullText)

Пермь, 2018 год 
 Аннотация В этой работе описан процесс реализации виртуального собеседника с помощью лексико-синтаксических шаблонов с использованием языка LSPL.  Для разработанного виртуального собеседника реализована система администрирования. Работа содержит 3 главы, 13 иллюстраций и 4 приложения.   
 Оглавление                            Введение Современное продуктивное обучение предполагает интенсивную работу как со стороны студентов, так и со стороны преподавателей, в том числе и выполнение большого количества проектов для усвоения полученных знаний на практике. Зачастую такие проекты требуют больших затрат времени как со стороны преподавателя, так и со стороны студентов. В этом случае такие проекты должны быть автоматизированы, хотя бы частично. Об автоматизации одного из таких учебных проектов и пойдет речь в этой работе.  В ходе работы будет улучшен разработанный ранее виртуальный собеседник для деловой игры «Развертывание» по технологии MSF. В ходе деловой игры студенты де

In [37]:
vkr = pd.read_csv('/content/vkr.csv?token=AITWEH753HPJ6QHZPBGT47LAETPYE')
#vkrText = pd.read_csv('/content/vkrText.csv?token=AITWEHZGZ5SB7PHMJA3YNTDAFFJOM')
vkrText.head()

,Unnamed: 0,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file,fullText,shortText
0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,,,
1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...,,
2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,,,
3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,,,
4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,,,


In [17]:
vkrText.head()

,Unnamed: 0,link_vkr,name_vkr,name_student,name_prof,campus,programm,grade,year,link_file,fullText,shortText
0,0,https://www.hse.ru/edu/vkr/219524501,Оценка стоимости компании,Дроздова Ирина Валерьевна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),-1,2018,NaN,,
1,1,https://www.hse.ru/edu/vkr/219524122,Оценка эффективности инвестиционного проекта,Стерледева Наталья Владимировна,Абашева Александра Сергеевна,Вечерне-заочный факультет экономики и управле...,Экономика (в) (Бакалавриат),7,2018,http://lms.hse.ru/ap_service.php?getwork=1&gui...,,
2,2,https://www.hse.ru/edu/vkr/153009903,Разработка системы управления знаниями в орган...,Беленова Анастасия Александровна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,,
3,3,https://www.hse.ru/edu/vkr/153009689,Брендинг инновационного продукта,Воронков Кирилл Евгеньевич,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,,
4,4,https://www.hse.ru/edu/vkr/153009541,Стратегия инновационного развития бизнеса в ин...,Бадягина Мария Германовна,Аленина Карина Анатольевна,Факультет менеджмента (Пермь),Менеджмент (Бакалавриат),-1,2015,NaN,,


In [16]:
vkrText = vkr.copy()
vkrText['fullText'] = ''
vkrText['shortText'] = ''

In [40]:
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        try:
            open('test.doc', 'wb').write(r.content)
            f = open('test.doc', 'rb')
            document = Document(f)
            f.close()
            successful.append(url)
        except:
            unsuccessful.append(url)
            vkrText.loc[row, 'fullText' ] = ''
            vkrText.loc[row, 'shortText'] = ''
            continue
        
        fullText = ""
        for para in document.paragraphs:
            fullText+=(para.text)+" "
        vkrText.loc[row, 'fullText' ] = fullText
        vkrText.loc[row, 'shortText'] = ''
        '''
        text = clearTextAfterGarbage(fullText)
        result = modelSum(fullText, min_length=20, max_length=100)
        summary = "".join(result)

        vkrText.loc[row, 'shortText'] = summary
        '''
    else:
        vkrText.loc[row, 'link_file'] = ''
        vkrText.loc[row, 'fullText' ] = ''
        vkrText.loc[row, 'shortText'] = ''

100%|██████████| 1708/1708 [03:41<00:00,  7.70it/s]


In [41]:
len(successful), len(unsuccessful)

(0, 72)

In [20]:
vkrText.to_csv('vkrText.csv')

In [52]:
tmp = vkrText[vkrText['link_file']!='']
tmp[tmp['fullText']=='']
tmp

In [73]:
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        if (vkrText.loc[row,'fullText'])!='':
            continue
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        try:
            open('test.doc', 'wb').write(r.content)
            vkrText.loc[row, 'fullText' ] = textract.process("/content/test.doc").decode("utf-8")
            successful.append(url)
        except:
            unsuccessful.append(url)
            continue

100%|██████████| 1708/1708 [04:33<00:00,  6.24it/s]


In [74]:
len(successful), len(unsuccessful)

(22, 50)

In [83]:
successful = []
unsuccessful = []
for row in tqdm(range(vkr.shape[0])):
    if type(vkr.loc[row, 'link_file'])==str:
        if (vkrText.loc[row,'fullText'])!='':
            continue
        url = vkr.loc[row, 'link_file']
        r = requests.get(url, allow_redirects=True)
        open('test.pdf', 'wb').write(r.content)
        try:
            vkrText.loc[row, 'fullText'] = textract.process("/content/test.pdf").decode("utf-8")
        except:
            unsuccessful.append(url)




  0%|          | 0/1708 [00:00<?, ?it/s]


 17%|█▋        | 290/1708 [00:03<00:19, 74.05it/s]


 17%|█▋        | 290/1708 [00:17<00:19, 74.05it/s]


 17%|█▋        | 295/1708 [00:22<27:09,  1.15s/it]


 20%|██        | 348/1708 [00:33<19:42,  1.15it/s]


 26%|██▌       | 443/1708 [07:37<41:00,  1.95s/it]


 28%|██▊       | 473/1708 [07:54<31:38,  1.54s/it]


 29%|██▉       | 496/1708 [08:11<26:10,  1.30s/it]


 29%|██▉       | 497/1708 [08:28<2:00:26,  5.97s/it]


 29%|██▉       | 498/1708 [08:50<3:37:14, 10.77s/it]


 30%|██▉       | 508/1708 [09:04<2:39:09,  7.96s/it]


 35%|███▌      | 605/1708 [09:18<1:43:11,  5.61s/it]


 38%|███▊      | 643/1708 [09:30<1:11:27,  4.03s/it]


 42%|████▏     | 710/1708 [09:39<47:35,  2.86s/it]  


 43%|████▎     | 739/1708 [09:47<33:33,  2.08s/it]


 46%|████▌     | 782/1708 [10:13<25:18,  1.64s/it]


 46%|████▌     | 783/1708 [11:22<5:36:12, 21.81s/it]


 51%|█████     | 863/1708 [11:37<3:35:46, 15.32s/it]


 51%|█████     | 870/1708 [11:56<2:41

In [84]:
vkrText.to_csv('vkrText.csv')

In [96]:
len(unsuccessful)

20

In [5]:
custom_config = AutoConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)

In [6]:
def lowerCase(Corpus):
    corpus = "".join([i.lower().replace('\n',' ') for i in Corpus])
    return corpus

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(Corpus):
    """custom function to remove the punctuation"""
    return "".join([i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) for i in Corpus])

def remove_stopwords(Corpus):
    """custom function to remove the stopwords"""
    return " ".join([token for token in Corpus.split() if token not in russian_stopwords])

def remove_freqwords(Corpus):
    cnt = Counter()
    for text in Corpus:
        for word in text.split():
            cnt[word] += 1
    FREQWORDS = set([w for (w, wc) in cnt.most_common(30)])
    return " ".join([word for word in str(Corpus).split() if word not in FREQWORDS])

def remove_urls(Corpus):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    return_text=[]
    for text in Corpus:
        text=url_pattern.sub(r'', text)
        text=html_pattern.sub(r'',text)
        return_text.append(text)
    return "".join(i for i in return_text)
    
def remove_numbers(Corpus):
    text = ["".join(i for i in text if not i.isdigit()) for text in Corpus]
    return "".join(i for i in text)

def Stem(Corpus):
    stemmer = SnowballStemmer("russian") 
    return "".join(m.lemmatize(Corpus))

def Lemm(Corpus):
    m = Mystem()
    return "".join(m.lemmatize(Corpus))

def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 

def clearTextAfterGarbage(Corpus):
    corpus = Corpus
    corpus = lowerCase(Corpus)
    corpus = remove_urls(corpus)
    corpus = remove_numbers(corpus)
    corpus = remove_stopwords(corpus)
    #corpus = remove_punctuation(corpus)
    corpus = Lemm(corpus)
    #corpus = Stem(corpus)
    corpus = remove_freqwords(corpus)
    return corpus

In [ ]:
text = clearTextAfterGarbage(fullText)
text

'пермь, год аннотация работа описывать процесс реализация виртуальный собеседник помощь лексико-синтаксический шаблон использование язык lspl. разрабатывать виртуальный собеседник реализовать система администрирование. работа содержать глава, иллюстрация приложение. оглавление введение современный продуктивный обучение предполагать интенсивный работа сторона студент, сторона преподаватель, число выполнение большой количество проект усвоение получать знание практика. зачастую такой проект требовать большой затрата время сторона преподаватель, сторона студент. случай такой проект должный автоматизировать, хотя частично. автоматизация один такой учебный проект пойти речь работа. ход работа улучшать разрабатывать ранее виртуальный собеседник деловой игра «развертывание» технология msf. ход деловой игра студент делиться группа, группа должный опрашивать сотрудник отдел фирма филиал четыре город, разбрасывать весь мир. каждый сотрудник обладать свой компетенция знать весь информация филиал, 

In [ ]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = modelSum(fullText, min_length=20, max_length=100)
summary = "".join(result)
print(len(summary), summary)

3115 Для разработанного виртуального собеседника реализована система администрирования. Актуальность работы: работа является актуальной, т.к. Объект работы: процесс проведения. Предмет работы: автоматизация консультационной работы преподавателя в ходе проекта. Первая глава содержит описание предметной области и анализ автоматизируемого процесса. Planning (Планирование). На этапе разработки пишется код приложения, подготавливается документация. Stabilizing (Стабилизация). В ходе игры студенты должны установить вымышленную систему «ОбалдеИТ» на компьютеры пользователей. Формирование расписания развертывания. Трудозатраты преподавателя. Выявление наличия в указанном филиале фирмы сотрудника с указанной должностью. Определение ответа сотрудника на заданный вопрос. Возможность просмотра заданных вопросов. Полный список требований к системе представлен в приложении А. Глава 2. Искусственные нейронные сети (ИНС). Попытка использования первой версии собеседника выявила следующие проблемы:  Кол

In [ ]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
result = modelSum(text, min_length=20, max_length=100)
summary = "".join(result)
print(len(summary), summary)

2525 разрабатывать виртуальный собеседник реализовать система администрирование. преподаватель это трудозатраты, студент - затрата время игра целое это длительность. работа разделять глава. developing (разработка). этап проводиться тестирование приложение, устраняться обнаруживать ошибка. ход игра студент должный устанавливать вымышленный система «обалдеит» компьютер пользователь. анализ игра разбирать процесс проведение игра рамка обучение студент. общение преподаватель одновременно доступно сразу несколько группа студент. трудозатраты преподаватель. преподаватель вынуждать отвечать весь студент многочисленный вопрос, требовать значительный усилие. готовый входной файл содержать запись. практика вариант пробовать реализовать. исходить следствие теорема арнольд–колмогоров-хехт–нильсен[], большой вероятность переобучение инс. лексико-синтаксический шаблон (лсш). русский язык это падеж, время, наклонение, склонение, число т.д. попытка использование первый версия собеседник выявлять следу

In [ ]:
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)
for row in tqdm(range(vkrText.shape[0])):
    if type(vkrText.loc[row, 'fullText'])!='':
        text = clearTextAfterGarbage(vkrText.loc[row, 'fullText'])
        result = modelSum(fullText, min_length=20, max_length=100)
        summary = "".join(result)

        vkrText.loc[row, 'shortText'] = summary